# experiment_4_k

In [2]:
!pip install kaggle wandb onnx -Uq
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
! mkdir ~/.kaggle

In [4]:
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting

  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 618MB/s]


In [7]:
! unzip walmart-recruiting-store-sales-forecasting.zip

Archive:  walmart-recruiting-store-sales-forecasting.zip
  inflating: features.csv.zip        
  inflating: sampleSubmission.csv.zip  
  inflating: stores.csv              
  inflating: test.csv.zip            
  inflating: train.csv.zip           


In [17]:
# ================================================================================
# EXPERIMENT 4: FIXING DATA LEAKAGE - SPLIT FIRST, THEN PREPROCESS
# ================================================================================

# Step 1: Setup and MLflow/DagsHub Configuration
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Install required packages
!pip install prophet plotly mlflow dagshub xgboost -q

# Setup MLflow and DagsHub
import mlflow
import dagshub

# DagsHub setup
dagshub.init(repo_owner='konstantine25b',
             repo_name='Walmart-Recruiting---Store-Sales-Forecasting',
             mlflow=True)

# Set tracking URI
mlflow.set_tracking_uri("https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow")
mlflow.set_experiment("Experiment_4_Fixed_Data_Leakage")

Initialized MLflow to track repo "konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting"

Repository konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting initialized!

<Experiment: artifact_location='mlflow-artifacts:/43ce0a29767b4be4b47a7e6d431382c2', creation_time=1750844432684, experiment_id='3', last_update_time=1750844432684, lifecycle_stage='active', name='Experiment_4_Fixed_Data_Leakage', tags={}>

In [53]:
with mlflow.start_run(run_name="Data_Loading_Fixed") as run:
    # Load datasets
    with zipfile.ZipFile('train.csv.zip', 'r') as zip_ref:
        zip_ref.extractall()

    train = pd.read_csv('train.csv')
    stores = pd.read_csv('stores.csv')

    # Convert Date column
    train['Date'] = pd.to_datetime(train['Date'])

    # Merge with stores data
    train_merged = train.merge(stores, on='Store', how='left')

    print(f"✅ Data loaded: {train_merged.shape}")
    print(f"📅 Date range: {train['Date'].min()} to {train['Date'].max()}")


✅ Data loaded: (421570, 7)
📅 Date range: 2010-02-05 00:00:00 to 2012-10-26 00:00:00
🏃 View run Data_Loading_Fixed at: https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/3/runs/df0f0eea88e0441b98208cbed624b5bc
🧪 View experiment at: https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/3


In [54]:
# STEP 2: SINGLE PROPER TEMPORAL SPLIT (BEFORE ANY PREPROCESSING)
# ================================================================================
with mlflow.start_run(run_name="Proper_Temporal_Split_Fixed") as run:

    print(f"🔪 PROPER TEMPORAL SPLIT - NO DATA LEAKAGE")

    # Sort data chronologically
    data_sorted = train_merged.sort_values(['Store', 'Dept', 'Date']).reset_index(drop=True)

    # Use SINGLE temporal split - 80% of TIME for training
    min_date = data_sorted['Date'].min()
    max_date = data_sorted['Date'].max()
    total_days = (max_date - min_date).days
    split_days = int(total_days * 0.8)
    split_date = min_date + timedelta(days=split_days)

    # Align to Friday (weekly data)
    while split_date.weekday() != 4:
        split_date += timedelta(days=1)

    # Create the split
    train_data = data_sorted[data_sorted['Date'] < split_date].copy()
    val_data = data_sorted[data_sorted['Date'] >= split_date].copy()

    print(f"📅 Split date: {split_date}")
    print(f"🚂 Training: {len(train_data):,} records")
    print(f"🔮 Validation: {len(val_data):,} records")
    print(f"📊 Training date range: {train_data['Date'].min()} to {train_data['Date'].max()}")
    print(f"📊 Validation date range: {val_data['Date'].min()} to {val_data['Date'].max()}")

    # Verify no data leakage
    assert train_data['Date'].max() < val_data['Date'].min(), "❌ DATA LEAKAGE DETECTED!"
    print("✅ No temporal data leakage - training ends before validation starts")

🔪 PROPER TEMPORAL SPLIT - NO DATA LEAKAGE
📅 Split date: 2012-04-13 00:00:00
🚂 Training: 335,761 records
🔮 Validation: 85,809 records
📊 Training date range: 2010-02-05 00:00:00 to 2012-04-06 00:00:00
📊 Validation date range: 2012-04-13 00:00:00 to 2012-10-26 00:00:00
✅ No temporal data leakage - training ends before validation starts
🏃 View run Proper_Temporal_Split_Fixed at: https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/3/runs/f40a3a2fde6148e9b782891c64b37a42
🧪 View experiment at: https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/3


In [55]:
# ================================================================================
# STEP 3: OUTLIER REMOVAL (ONLY ON TRAINING DATA)
# ================================================================================
with mlflow.start_run(run_name="Outlier_Removal_Training_Only") as run:

    print(f"🔍 OUTLIER REMOVAL - TRAINING DATA ONLY")

    # Calculate outlier thresholds ONLY on training data
    train_sales_by_group = train_data.groupby(['Store', 'Dept'])['Weekly_Sales']
    Q1_train = train_sales_by_group.transform(lambda x: x.quantile(0.25))
    Q3_train = train_sales_by_group.transform(lambda x: x.quantile(0.75))
    IQR_train = Q3_train - Q1_train

    lower_bound = Q1_train - 1.5 * IQR_train
    upper_bound = Q3_train + 1.5 * IQR_train

    # Find outliers in training data
    outliers_mask = (train_data['Weekly_Sales'] < lower_bound) | (train_data['Weekly_Sales'] > upper_bound)
    outliers_count = outliers_mask.sum()

    # Remove outliers from training data ONLY
    train_data_clean = train_data[~outliers_mask].copy()
    # Keep validation data unchanged
    val_data_clean = val_data.copy()

    print(f"🗑️ Removed {outliers_count:,} outliers from training data ({outliers_count/len(train_data)*100:.1f}%)")
    print(f"🚂 Training after outlier removal: {len(train_data_clean):,}")
    print(f"🔮 Validation unchanged: {len(val_data_clean):,}")


🔍 OUTLIER REMOVAL - TRAINING DATA ONLY
🗑️ Removed 15,467 outliers from training data (4.6%)
🚂 Training after outlier removal: 320,294
🔮 Validation unchanged: 85,809
🏃 View run Outlier_Removal_Training_Only at: https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/3/runs/f2aa6279169f4c4d93f1346dee918634
🧪 View experiment at: https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/3


In [56]:
# ================================================================================
# STEP 4: FEATURE ENGINEERING (APPLIED SEPARATELY)
# ================================================================================
with mlflow.start_run(run_name="Feature_Engineering_Fixed") as run:

    print(f"🔧 FEATURE ENGINEERING - APPLIED SEPARATELY TO EACH SET")

    def create_date_features(df):
        """Create basic date features"""
        df = df.copy()
        df['Year'] = df['Date'].dt.year
        df['Month'] = df['Date'].dt.month
        df['Quarter'] = df['Date'].dt.quarter
        df['DayOfWeek'] = df['Date'].dt.dayofweek
        df['Week'] = df['Date'].dt.isocalendar().week

        # Cyclical features
        df['Month_sin'] = np.sin(2 * np.pi * df['Month'] / 12)
        df['Month_cos'] = np.cos(2 * np.pi * df['Month'] / 12)

        # Days from start
        reference_date = pd.Timestamp('2010-02-05')
        df['DaysFromStart'] = (df['Date'] - reference_date).dt.days

        # Holiday features
        df['IsHolidayMonth'] = df['Month'].isin([11, 12]).astype(int)
        return df

    def create_lag_features(df, target_col='Weekly_Sales'):
        """Create lag features without data leakage"""
        df = df.copy()
        df = df.sort_values(['Store', 'Dept', 'Date']).reset_index(drop=True)

        # Simple lag features
        for lag in [1, 2, 4]:
            lag_col = f'{target_col}_lag_{lag}'
            df[lag_col] = df.groupby(['Store', 'Dept'])[target_col].shift(lag)

        # Fill NaNs with 0 (for early periods without enough history)
        lag_cols = [col for col in df.columns if 'lag_' in col]
        for col in lag_cols:
            df[col] = df[col].fillna(0)

        return df

    # Apply feature engineering to each set separately
    print("🚂 Creating features for training set...")
    train_features = create_date_features(train_data_clean)
    train_final = create_lag_features(train_features)

    print("🔮 Creating features for validation set...")
    val_features = create_date_features(val_data_clean)
    val_final = create_lag_features(val_features)

    # Ensure same columns
    common_cols = list(set(train_final.columns).intersection(set(val_final.columns)))
    train_final = train_final[common_cols]
    val_final = val_final[common_cols]

    print(f"✅ Feature engineering completed")
    print(f"🚂 Training shape: {train_final.shape}")
    print(f"🔮 Validation shape: {val_final.shape}")
    print(f"📊 Common features: {len(common_cols)}")


🔧 FEATURE ENGINEERING - APPLIED SEPARATELY TO EACH SET
🚂 Creating features for training set...
🔮 Creating features for validation set...
✅ Feature engineering completed
🚂 Training shape: (320294, 19)
🔮 Validation shape: (85809, 19)
📊 Common features: 19
🏃 View run Feature_Engineering_Fixed at: https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/3/runs/c31b52911d8341e9a90ed105050da1d8
🧪 View experiment at: https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/3


In [57]:

# ================================================================================
# STEP 5: PREPARE DATA FOR TRAINING (CRITICAL FIX)
# ================================================================================
with mlflow.start_run(run_name="Data_Preparation_Fixed") as run:

    print(f"📊 PREPARING DATA FOR TRAINING - CRITICAL VERIFICATION")

    # Define features and target
    target_col = 'Weekly_Sales'
    exclude_cols = ['Weekly_Sales', 'Date']
    feature_cols = [col for col in train_final.columns if col not in exclude_cols]

    # Prepare training data
    X_train = train_final[feature_cols].copy()
    y_train = train_final[target_col].copy()
    train_is_holiday = train_final['IsHoliday'].copy()

    # Prepare validation data
    X_val = val_final[feature_cols].copy()
    y_val = val_final[target_col].copy()
    val_is_holiday = val_final['IsHoliday'].copy()

    # CRITICAL VERIFICATION - PREVENT DATA SHAPE CHANGES
    assert len(X_train) == len(train_data_clean), f"❌ Training data size changed! Expected {len(train_data_clean)}, got {len(X_train)}"
    assert len(X_val) == len(val_data_clean), f"❌ Validation data size changed! Expected {len(val_data_clean)}, got {len(X_val)}"

    print(f"✅ Data shape verification passed!")
    print(f"🚂 X_train: {X_train.shape}")
    print(f"🚂 y_train: {y_train.shape}")
    print(f"🔮 X_val: {X_val.shape}")
    print(f"🔮 y_val: {y_val.shape}")
    print(f"📊 Features: {len(feature_cols)}")

    # Handle any remaining NaNs
    if X_train.isnull().sum().sum() > 0 or X_val.isnull().sum().sum() > 0:
        print("⚠️ Filling remaining NaNs...")
        X_train = X_train.fillna(0)
        X_val = X_val.fillna(0)


📊 PREPARING DATA FOR TRAINING - CRITICAL VERIFICATION
✅ Data shape verification passed!
🚂 X_train: (320294, 17)
🚂 y_train: (320294,)
🔮 X_val: (85809, 17)
🔮 y_val: (85809,)
📊 Features: 17
🏃 View run Data_Preparation_Fixed at: https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/3/runs/80d1164881464557addf982a51a4d80e
🧪 View experiment at: https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/3


In [59]:
# ================================================================================
# STEP 6: DATA TYPE FIX BEFORE XGBOOST TRAINING
# ================================================================================
with mlflow.start_run(run_name="Data_Type_Fix_Before_XGBoost") as run:

    print(f"🔧 FIXING DATA TYPES FOR XGBOOST")

    # Check data types
    print("📊 Checking data types...")
    print("Training data types:")
    print(X_train.dtypes.value_counts())
    print("\nValidation data types:")
    print(X_val.dtypes.value_counts())

    # Find problematic columns
    object_cols = []
    for col in X_train.columns:
        if X_train[col].dtype == 'object' or X_val[col].dtype == 'object':
            object_cols.append(col)
            print(f"   Found object column: {col}")

    # Fix categorical columns with one-hot encoding
    if object_cols:
        print(f"🔧 One-hot encoding {len(object_cols)} categorical columns...")

        for col in object_cols:
            print(f"   Encoding {col}...")

            # Combine train and val for consistent encoding
            combined_data = pd.concat([
                X_train[col].reset_index(drop=True),
                X_val[col].reset_index(drop=True)
            ])

            # Create dummy variables
            dummies = pd.get_dummies(combined_data, prefix=col, dummy_na=False)

            # Split back
            train_dummies = dummies.iloc[:len(X_train)].copy()
            val_dummies = dummies.iloc[len(X_train):].copy()

            # Reset indices to match original data
            train_dummies.index = X_train.index
            val_dummies.index = X_val.index

            # Add dummy columns and remove original
            X_train = pd.concat([X_train.drop(columns=[col]), train_dummies], axis=1)
            X_val = pd.concat([X_val.drop(columns=[col]), val_dummies], axis=1)

            print(f"     Created {len(dummies.columns)} dummy variables")

    # Convert any remaining non-numeric columns
    print("🔧 Converting remaining columns to numeric...")

    for col in X_train.columns:
        if X_train[col].dtype not in ['int64', 'float64', 'int32', 'float32', 'bool']:
            print(f"   Converting {col} from {X_train[col].dtype}")
            X_train[col] = pd.to_numeric(X_train[col], errors='coerce')
            X_val[col] = pd.to_numeric(X_val[col], errors='coerce')

    # Convert boolean to int
    bool_cols = []
    for col in X_train.columns:
        if X_train[col].dtype == 'bool':
            bool_cols.append(col)
            X_train[col] = X_train[col].astype(int)
            X_val[col] = X_val[col].astype(int)

    if bool_cols:
        print(f"   Converted {len(bool_cols)} boolean columns to int")

    # Fill any NaNs introduced during conversion
    train_nans = X_train.isnull().sum().sum()
    val_nans = X_val.isnull().sum().sum()

    if train_nans > 0 or val_nans > 0:
        print(f"⚠️ Filling NaNs: Train={train_nans}, Val={val_nans}")
        X_train = X_train.fillna(0)
        X_val = X_val.fillna(0)

    # Final verification
    print(f"\n✅ Final data type check:")
    print(f"   Training data types: {X_train.dtypes.value_counts().to_dict()}")
    print(f"   Validation data types: {X_val.dtypes.value_counts().to_dict()}")
    print(f"   Training shape: {X_train.shape}")
    print(f"   Validation shape: {X_val.shape}")

    # Verify no object columns remain
    train_objects = [col for col in X_train.columns if X_train[col].dtype == 'object']
    val_objects = [col for col in X_val.columns if X_val[col].dtype == 'object']

    if train_objects or val_objects:
        print(f"❌ Still have object columns: Train={train_objects}, Val={val_objects}")
        raise ValueError("Object columns still exist!")
    else:
        print(f"✅ All columns are now numeric!")


🔧 FIXING DATA TYPES FOR XGBOOST
📊 Checking data types...
Training data types:
float64    5
int64      5
int32      4
bool       1
object     1
UInt32     1
Name: count, dtype: int64

Validation data types:
float64    5
int64      5
int32      4
bool       1
object     1
UInt32     1
Name: count, dtype: int64
   Found object column: Type
🔧 One-hot encoding 1 categorical columns...
   Encoding Type...
     Created 3 dummy variables
🔧 Converting remaining columns to numeric...
   Converting Week from UInt32
   Converted 4 boolean columns to int

✅ Final data type check:
   Training data types: {dtype('int64'): 9, dtype('float64'): 5, dtype('int32'): 4, UInt32Dtype(): 1}
   Validation data types: {dtype('int64'): 9, dtype('float64'): 5, dtype('int32'): 4, UInt32Dtype(): 1}
   Training shape: (320294, 19)
   Validation shape: (85809, 19)
✅ All columns are now numeric!
🏃 View run Data_Type_Fix_Before_XGBoost at: https://dagshub.com/konstantine25b/Walmart-Recruiting---Store-Sales-Forecasting.

In [61]:
# ================================================================================
# STEP 7: XGBOOST TRAINING (AFTER DATA TYPE FIX)
# ================================================================================
with mlflow.start_run(run_name="XGBoost_Training_PROPERLY_Fixed") as run:

    print(f"🚀 XGBOOST TRAINING - PROPERLY FIXED VERSION")

    # Define WMAE function
    def calculate_wmae(y_true, y_pred, is_holiday, holiday_weight=5.0):
        abs_errors = np.abs(y_true - y_pred)
        weights = np.where(is_holiday, holiday_weight, 1.0)
        wmae = np.sum(weights * abs_errors) / np.sum(weights)
        return wmae

    # XGBoost parameters (less aggressive to prevent overfitting)
    xgb_params = {
        'n_estimators': 500,      # Reduced from 1000
        'max_depth': 6,           # Reduced from 8
        'learning_rate': 0.05,    # Reduced from 0.1
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'reg_alpha': 1.0,         # Increased regularization
        'reg_lambda': 2.0,        # Increased regularization
        'random_state': 42,
        'n_jobs': -1,
        'objective': 'reg:squarederror'
    }

    # Train model
    print("🤖 Training XGBoost model...")
    xgb_model = xgb.XGBRegressor(**xgb_params)
    xgb_model.fit(X_train, y_train)

    # Make predictions
    print("🔮 Making predictions...")
    train_pred = xgb_model.predict(X_train)
    val_pred = xgb_model.predict(X_val)

    # Calculate metrics
    print("📊 Calculating metrics...")

    # Training metrics
    train_mae = mean_absolute_error(y_train, train_pred)
    train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
    train_r2 = r2_score(y_train, train_pred)
    train_wmae = calculate_wmae(y_train, train_pred, train_is_holiday)

    # Validation metrics
    val_mae = mean_absolute_error(y_val, val_pred)
    val_rmse = np.sqrt(mean_squared_error(y_val, val_pred))
    val_r2 = r2_score(y_val, val_pred)
    val_wmae = calculate_wmae(y_val, val_pred, val_is_holiday)

    # Display results
    print(f"\n" + "="*80)
    print(f"🎯 PROPERLY FIXED EXPERIMENT 4 RESULTS")
    print(f"="*80)

    print(f"\n🚂 Training Metrics:")
    print(f"   WMAE: ${train_wmae:,.2f}")
    print(f"   MAE: ${train_mae:,.2f}")
    print(f"   RMSE: ${train_rmse:,.2f}")
    print(f"   R²: {train_r2:.4f}")

    print(f"\n🔮 Validation Metrics:")
    print(f"   WMAE: ${val_wmae:,.2f}")
    print(f"   MAE: ${val_mae:,.2f}")
    print(f"   RMSE: ${val_rmse:,.2f}")
    print(f"   R²: {val_r2:.4f}")

    # Overfitting check
    wmae_diff = abs(train_wmae - val_wmae) / val_wmae * 100
    print(f"\n📈 Overfitting Analysis:")
    print(f"   WMAE difference: {wmae_diff:.1f}%")
    if wmae_diff > 50:
        print(f"   ⚠️ Significant overfitting detected!")
    elif wmae_diff > 20:
        print(f"   ⚠️ Moderate overfitting detected")
    else:
        print(f"   ✅ Reasonable generalization")

    # Feature importance
    print(f"\n🎯 Top 10 Feature Importance:")
    feature_importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': xgb_model.feature_importances_
    }).sort_values('importance', ascending=False)

    for i, (_, row) in enumerate(feature_importance.head(10).iterrows()):
        print(f"   {i+1:2d}. {row['feature']:25s}: {row['importance']:.4f}")

    # Log metrics to MLflow
    mlflow.log_metric("train_wmae", train_wmae)
    mlflow.log_metric("val_wmae", val_wmae)
    mlflow.log_metric("train_mae", train_mae)
    mlflow.log_metric("val_mae", val_mae)
    mlflow.log_metric("train_r2", train_r2)
    mlflow.log_metric("val_r2", val_r2)
    mlflow.log_metric("overfitting_percentage", wmae_diff)

    for param, value in xgb_params.items():
        mlflow.log_param(f"xgb_{param}", value)

    print(f"\n" + "="*80)
    print(f"🎉 EXPERIMENT 4 PROPERLY COMPLETED!")
    print(f"✅ Data Leakage: ELIMINATED")
    print(f"🎯 Validation WMAE: ${val_wmae:,.2f}")
    print(f"📊 This should be more realistic (~$2000-3000 range)")
    print(f"="*80)

🚀 XGBOOST TRAINING - PROPERLY FIXED VERSION
🤖 Training XGBoost model...
🔮 Making predictions...
📊 Calculating metrics...

🎯 PROPERLY FIXED EXPERIMENT 4 RESULTS

🚂 Training Metrics:
   WMAE: $1,338.89
   MAE: $1,267.53
   RMSE: $2,495.64
   R²: 0.9868

🔮 Validation Metrics:
   WMAE: $2,519.10
   MAE: $2,544.79
   RMSE: $5,669.32
   R²: 0.9333

📈 Overfitting Analysis:
   WMAE difference: 46.9%
   ⚠️ Moderate overfitting detected

🎯 Top 10 Feature Importance:
    1. Weekly_Sales_lag_1       : 0.6347
    2. Weekly_Sales_lag_2       : 0.2868
    3. Weekly_Sales_lag_4       : 0.0345
    4. Month                    : 0.0072
    5. DaysFromStart            : 0.0056
    6. Dept                     : 0.0054
    7. Type_A                   : 0.0048
    8. Type_B                   : 0.0034
    9. Size                     : 0.0028
   10. Week                     : 0.0027

🎉 EXPERIMENT 4 PROPERLY COMPLETED!
✅ Data Leakage: ELIMINATED
🎯 Validation WMAE: $2,519.10
📊 This should be more realistic (~$20